In [1]:
# use magic command to auto-reload
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv

sys.path.insert(0, os.path.join(os.getcwd(), '..'))  # parent folder of src
from src.simulation import run_simulation
from src.save import save_simulation_results, show_discussion_from_saved, load_replica
from src.visualization import show_interactive_conversation_tree_with_agent_selection
import dotenv
import os
import asyncio
import aiohttp


In [3]:
# load PARAMS.json
with open("PARAMS.json", "r") as f:
    PARAMS = json.load(f)

PARAMS["time_decay_rate"] =  np.log(2)/2  # halving the weight every 2 time steps. change /2 -> /k for halving every k time steps.
PARAMS["W_agent_success"] = 1
PARAMS["W_personal_weights"] = 1
PARAMS["W_post_success"] = 1
PARAMS["noise_level"] = 0.01



ENV = dotenv.dotenv_values("./.env")
HF_TOKEN = ENV["HF_TOKEN"]
API_URL = ENV["API_URL"]
MODEL = ENV["MODEL"]


print(API_URL)
print(MODEL)

# add HF_TOKEN to PARAMS
PARAMS["HF_TOKEN"] = HF_TOKEN
PARAMS["API_URL"] = API_URL
PARAMS["MODEL"] = MODEL



timesteps = 5
num_agents = 50
fill_history = 1


PARAMS["num_agents"] = num_agents
PARAMS["timesteps"] = timesteps
PARAMS["fill_history"] = fill_history

https://router.huggingface.co/v1/chat/completions
meta-llama/Llama-3.1-8B-Instruct:cerebras


In [4]:
List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES, List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES  = await run_simulation(PARAMS)

T_current: 1
[reader_id, sender_id] = [0, 38]
prompt: [{'role': 'system', 'content': 'You are a person who thinks and writes like this:\n            - ""Can we really afford universal healthcare?  I\'ve seen the stats and I\'m not convinced it\'s the right plan for America. The idea of free healthcare sounds great, but someone\'s gotta foot the bill. And let me tell you, it\'s not going to be rich businessmen buying private insurance. It\'s gonna be small business owners, families, and workers like you and me. \n\nAnd what about the people who abuse the system? Freeloading on the rest of us while they collect a check without even contributing to it? It\'s not a moral solution for me. Personal responsibility is what matters. Either you take care of your health or you don\'t - no one owes you anything. Government can help with programs for the truly needy, but I don\'t think blanket coverage is the solution."\n\nNote: The tone of this post reflects the individual\'s personality traits an

In [5]:
out_root = "runs"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES, List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)

In [6]:

# Load a specific replica
replica_dir = "runs/r000"  # Replace with your actual path
replica_data = load_replica(replica_dir)

# Extract the individual likes data
individual_likes_data = replica_data["INDIVIDUAL_LIKES"]

# Check its structure
print("Shape:", individual_likes_data.shape)
print("Type:", type(individual_likes_data))

Shape: (50, 50, 6)
Type: <class 'numpy.ndarray'>


In [7]:
print("List shape:", List_of_INDIVIDUAL_LIKES[0].shape)
print("Loaded shape:", individual_likes_data.shape)
print("Are they equal?", np.array_equal(List_of_INDIVIDUAL_LIKES[0], individual_likes_data))

List shape: (50, 50, 6)
Loaded shape: (50, 50, 6)
Are they equal? True


In [10]:
70/50*5*50

350.0

In [8]:
fig = show_interactive_conversation_tree_with_agent_selection("runs/r000")


In [23]:
print(List_of_POSTS[0][3][0])

I recently realized that my grandparents struggled to manage their mounting medical bills even with retirement savings. A universal healthcare system could've brought them more peace of mind & financial stability during their golden years.
